In [1]:
import os
import sys
import time
import math

import dill
from tqdm import tqdm
import hyperopt as hpo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from src import utils
from src import model2 as m2
import src.dataset as dset
import src.pytorch_utils as ptu
import src.chu_liu_edmonds as chu

import warnings
warnings.filterwarnings('ignore')

seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(seed)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)
versions_dir = 'models'

cpu


In [2]:
# train_dataset = dset.DataSet('data/train.labeled', tqdm_bar=True, use_glove=True)
# test_dataset = dset.DataSet('data/test.labeled', train_dataset=train_dataset, tqdm_bar=True, use_glove=True)
# comp_dataset = dset.DataSet('data/comp.unlabeled', train_dataset=train_dataset, tagged=False, tqdm_bar=True, use_glove=True)

In [3]:
# with open(os.path.join('data', 'train_dataset.pth'), 'wb') as f:
#     dill.dump(train_dataset, f)
# with open(os.path.join('data', 'test_dataset.pth'), 'wb') as f:
#     dill.dump(test_dataset, f)

In [4]:
with open(os.path.join('data', 'train_dataset.pth'), "rb") as f:
    train_dataset = dill.load(f)
with open(os.path.join('data', 'test_dataset.pth'), "rb") as f:
    test_dataset = dill.load(f)

In [5]:
version = 'V2_1.4'
save = True

In [6]:
checkpoint = ptu.load_model(version=version, versions_dir=versions_dir, epoch='best', seed=42)
display(checkpoint.log)

model version: V2_1.4
Number of parameters 5882138 trainable 5882138
epochs: 43
train_time: 93.991

last train_loss: 0.13751
last val_loss: 0.75602
last train_score: 0.97208
last val_score: 0.89587
best val_score: 0.89587 at epoch 43


,train_time,timestamp,train_loss,val_loss,train_score,val_score,batch_size,best,amsgrad,betas,eps,lr,weight_decay
1,1.533699,22:45:40 24-06-2020,1.261505,1.244037,0.642290,0.647441,16,True,False,"(0.9, 0.999)",1.000000e-08,0.001000,5.000000e-07
2,3.773514,22:47:54 24-06-2020,0.748725,0.750303,0.790899,0.792847,16,True,False,"(0.9, 0.999)",1.000000e-08,0.001000,5.000000e-07
3,6.443358,22:50:34 24-06-2020,0.576586,0.601003,0.834809,0.833135,16,True,False,"(0.9, 0.999)",1.000000e-08,0.001000,5.000000e-07
4,8.837217,22:52:58 24-06-2020,0.465435,0.509036,0.863605,0.853731,16,True,False,"(0.9, 0.999)",1.000000e-08,0.001000,5.000000e-07
5,11.621812,22:55:45 24-06-2020,0.401955,0.468448,0.878303,0.862117,16,True,False,"(0.9, 0.999)",1.000000e-08,0.001000,5.000000e-07
6,13.149734,22:58:43 24-06-2020,0.341593,0.433814,0.894669,0.872477,16,True,False,"(0.9, 0.999)",1.000000e-08,0.000400,5.000000e-07
7,15.715373,23:01:17 24-06-2020,0.316257,0.425450,0.902167,0.874286,16,True,False,"(0.9, 0.999)",1.000000e-08,0.000384,5.000000e-07
8,18.031162,23:03:36 24-06-2020,0.297770,0.427326,0.908279,0.875683,16,True,False,"(0.9, 0.999)",1.000000e-08,0.000369,5.000000e-07
9,20.547501,23:06:07 24-06-2020,0.279406,0.427565,0.912895,0.876711,16,True,False,"(0.9, 0.999)",1.000000e-08,0.000354,5.000000e-07
10,23.033982,23:08:36 24-06-2020,0.268982,0.433699,0.916366,0.876999,16,True,False,"(0.9, 0.999)",1.000000e-08,0.000340,5.000000e-07


In [7]:
loss, score = checkpoint.predict(test_dataset.dataset,
                                 batch_size=32,
                                 device=device,
                                 results=False,
                                 decision_func=chu.test_chu_liu_edmonds)
print(f'chu_liu_edmonds_UAS: {score}')

chu_liu_edmonds_UAS: 0.8959095580678315


In [8]:
checkpoint.model.lstm.dropout

0.4

In [9]:
checkpoint.model.word_dropout

0.2